# Détection des bio-marqueurs pour le cancer colorectal

## Etudiants
- Potet Bastien
- Kot Chau Ying

In [24]:
import os
import numpy as np
import pylab as plt
%matplotlib inline
import pandas as pd

In [25]:
import numpy as np
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

records = load_geo('data/GSE21510_family.soft')

In [27]:
nr = next(records)
print(nr)
print('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
print(nr.entity_attributes)

GEO Type: SERIES
GEO Id: GSE21510
Series_contact_address: 2-1-1 Hongo

Series_contact_city: Bunkyo-ku

Series_contact_country: Japan

Series_contact_department: Intractable Disease Research Center

Series_contact_email: mogushi-k@umin.ac.jp

Series_contact_institute: Juntendo University

Series_contact_name: Kaoru,,Mogushi

Series_contact_phone: +81-3-5802-1797

Series_contact_state: Tokyo

Series_contact_zip/postal_code: 113-8421

Series_contributor: Shunsuke,,Tsukamoto

Series_contributor: Toshiaki,,Ishikawa

Series_contributor: Satoru,,Iida

Series_contributor: Megumi,,Ishiguro

Series_contributor: Kaoru,,Mogushi

Series_contributor: Hiroshi,,Mizushima

Series_contributor: Hiroyuki,,Uetake

Series_contributor: Hiroshi,,Tanaka

Series_contributor: Kenichi,,Sugihara

Series_geo_accession: GSE21510

Series_last_update_date: Mar 25 2019

Series_overall_design: Total of 148 microarray datasets obtaine
d from LCM and homogenized tissues of colorectal cancer patients were normalized
 using

In [28]:
# start by re-loading the data
records = load_geo('data/GSE21510_family.soft')


id_cond = {}

dict_gene = dict()

data = []

# loop over records
for r in records:

    rea = r.entity_attributes

    if 'Sample_characteristics_ch1' in rea :
        sample = rea['Sample_description'][1]
        desc = rea["Sample_characteristics_ch1"]
        id_cond[sample] = desc

    if 'sample_table_begin' in rea:
        data.append(r.table_rows)

    if 'platform_table_begin' in rea :
        for i in r.table_rows :
            dict_gene[i[0]] = i[1]


# converts data to an array (more practical to handle)
data = np.array(data)

In [29]:
metastatis_list = []
stage_list = []
tissue_list = []

for k in id_cond:
    v = id_cond[k]
    metastatis = v[0].split(':')[1].strip()
    stage = v[1].split(':')[1].strip()
    tissue = v[2].split(':')[1].split(',')[0].strip()
    
    metastatis_list.append(metastatis)
    stage_list.append(stage)
    tissue_list.append(tissue)
    
    id_cond[k] = [metastatis, stage, tissue]

metastatis_list = list(set(metastatis_list))
stage_list = list(set(stage_list))
tissue_list = list(set(tissue_list))

In [30]:
print(metastatis_list)
print(stage_list)
print(tissue_list)

['metastasis', 'none', 'metastatic recurrence']
['3c', '3a', '1', '2b', '3b', '2a', '4', '0']
['normal', 'cancer']


In [17]:
import random
k = 6
idx_test = random.sample(range(len(id_cond)), k)
idx_train = list(range(0, len(id_cond) + 2))
[idx_train.remove(i) for i in idx_test]

data_test = data[:, idx_test]
data = data[:, idx_train]

print(np.shape(data))
print(np.shape(data_test))

(148, 144, 2)
(148, 6, 2)


In [23]:
desired_profile = [dict_gene[s] for s in data[0,2:,0]]
print('desired_profile = '+str(desired_profile))

desired_profile = ['M87338', 'X51757', 'X69699', 'L36861', 'L13852', 'X55005', 'X79510', 'J02843', 'X75208', 'L38487', 'M33318', 'NM_005505', 'NM_015140', 'NM_052871', 'NM_080735', 'NM_138957', 'NM_138957', 'NM_145004', 'NM_138796', 'NM_153359', 'NM_153359', 'BC014479', 'BG573647', 'NM_080432', 'NM_080655', 'NM_080669', 'AK074161', 'NM_138379', 'NM_173596', 'NM_024786', 'NM_080653', 'NM_001132', 'BC034926', 'BC034926', 'NM_017861', 'NM_152774', 'NM_152264', 'NM_153274', 'NM_145025', 'NM_032854', 'NM_145041', 'NM_145041', 'NM_032834', 'BC033730', 'NM_153211', 'NM_144573', 'NM_144597', 'NM_032753', 'NM_005927', 'NM_172098', 'NM_130759', 'AK091818', 'NM_144506', 'NM_033124', 'NM_033124', 'NM_138819', 'NM_138819', 'NM_145020', 'NM_145020', 'BC029352', 'NM_145039', 'NM_138632', 'BG571343', 'NM_053054', 'NM_014621', 'AY177407', 'NM_152860', 'NM_054026', 'NM_145311', 'NM_152891', 'NM_152891', 'NM_152769', 'NM_152769', 'AL833526', 'NM_052971', 'BC001526', 'NM_033128', 'NM_080618', 'NM_173487',